In [1]:
import os

os.environ.setdefault('TORCH_COMPILE_DISABLE', '1')
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import torch

# Method 2: Patch torch._dynamo.disable decorator after import
try:
    import torch._dynamo
    # Patch the disable function to ignore the 'wrapping' parameter
    if hasattr(torch._dynamo, 'disable'):
        def patched_disable(fn=None, *args, **kwargs):
            # Remove problematic 'wrapping' parameter if present
            if 'wrapping' in kwargs:
                kwargs.pop('wrapping')
            if fn is None:
                # Decorator usage: @disable
                return lambda f: f
            # Function usage: disable(fn) or disable(fn, **kwargs)
            # Simply return the function unwrapped to avoid recursion
            # The original disable was causing issues, so we bypass it entirely
            return fn
        torch._dynamo.disable = patched_disable
except Exception as e:
    print(f"Warning: Could not patch torch._dynamo: {e}")
    pass  # If patching fails, continue anyway

import random, string

from torchtext import data , datasets
from collections import defaultdict, Counter
import numpy as np

os.environ['GENSIM_DATA_DIR'] = os.path.join(os.getcwd(), 'gensim-data')

import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_model

from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
import time, copy
from utils import *

In [2]:
SEED = 42
train_data, validation_data, test_data, LABEL, TEXT, pretrained_embed = data_prep(SEED)

[*] Prepping Data...
[+] Test set formed!
[+] Train and Validation sets formed!
[+] Data prepped successfully!
[*] Retrieving pretrained word embeddings...
[*] Loading fasttext model...
[+] Model loaded!
[*] Forming embedding matrix...
[+] Embedding matrix formed!
[+] Embeddings retrieved successfully!

Label distribution in training set:
- ABBR: 69 samples (1.58%)
- DESC: 930 samples (21.32%)
- ENTY: 1000 samples (22.93%)
- HUM: 978 samples (22.42%)
- LOC: 668 samples (15.31%)
- NUM: 717 samples (16.44%)
Total samples: 4362, Sum of percentages: 100.00%


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# Build vocabulary for labels
LABEL.build_vocab(train_data)
num_classes = len(LABEL.vocab)
print(f"\nNumber of classes: {num_classes}")
print(f"Classes: {LABEL.vocab.itos}")

# Get pretrained embeddings from Part 1 (frozen embeddings)
pretrained_embeddings = pretrained_embed.weight.data

# Get embedding dimension and vocab size from the fasttext embedding layer
embedding_dim = pretrained_embed.weight.shape[1]
embedding_vocab_size = pretrained_embed.weight.shape[0]  # Vocab size from saved embedding


Number of classes: 6
Classes: ['ENTY', 'HUM', 'DESC', 'NUM', 'LOC', 'ABBR']


In [5]:
print(f'TEXT Vocab Size: {len(TEXT.vocab.stoi)}')

TEXT Vocab Size: 8166


In [6]:
# ============================================================================
# PART 3.4: Targeted Improvement for Weak Topics
# Strategy: Data Augmentation, Positional Embeddings
# ============================================================================

print("\n" + "="*80)
print("PART 3.4: TARGETED IMPROVEMENT FOR WEAK TOPICS")
print("="*80)
print("\nStrategies:")
print("  1. Data Augmentation for imbalanced classes (especially ABBR)")
print("  2. Positional Embeddings in attention layer")
print("="*80)

# Import required libraries for augmentation
import nltk
from nltk.corpus import wordnet
try:
    nltk.download('wordnet', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
except:
    pass



PART 3.4: TARGETED IMPROVEMENT FOR WEAK TOPICS

Strategies:
  1. Data Augmentation for imbalanced classes (especially ABBR)
  2. Positional Embeddings in attention layer


In [7]:
# ============================================================================
# Step 2: Data Augmentation Functions for Imbalanced Classes
# ============================================================================

print("\n>>> Step 2: Implementing Data Augmentation Functions...")

def get_synonyms(word):
    """Get synonyms for a word using WordNet"""
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ').lower()
            if synonym != word and synonym.isalpha():
                synonyms.add(synonym)
    return list(synonyms)

def synonym_replacement(tokens, n=1):
    """Replace n random words with their synonyms"""
    new_tokens = tokens.copy()
    words_to_replace = [i for i, word in enumerate(tokens) if word.isalpha() and len(word) > 2]
    
    if len(words_to_replace) == 0:
        return tokens
    
    num_replacements = min(n, len(words_to_replace))
    indices_to_replace = random.sample(words_to_replace, num_replacements)
    
    for idx in indices_to_replace:
        synonyms = get_synonyms(tokens[idx])
        if synonyms:
            new_tokens[idx] = random.choice(synonyms)
    
    return new_tokens

def random_insertion(tokens, n=1):
    """Randomly insert synonyms of n words"""
    new_tokens = tokens.copy()
    
    for _ in range(n):
        if len(new_tokens) == 0:
            break
        word = random.choice(new_tokens)
        synonyms = get_synonyms(word)
        if synonyms:
            synonym = random.choice(synonyms)
            insert_pos = random.randint(0, len(new_tokens))
            new_tokens.insert(insert_pos, synonym)
    
    return new_tokens

def random_deletion(tokens, p=0.1):
    """Randomly delete words with probability p"""
    if len(tokens) == 1:
        return tokens
    
    new_tokens = []
    for token in tokens:
        if random.random() > p:
            new_tokens.append(token)
    
    if len(new_tokens) == 0:
        return tokens[:1]
    
    return new_tokens

def random_swap(tokens, n=1):
    """Randomly swap n pairs of words"""
    new_tokens = tokens.copy()
    
    for _ in range(n):
        if len(new_tokens) < 2:
            break
        idx1, idx2 = random.sample(range(len(new_tokens)), 2)
        new_tokens[idx1], new_tokens[idx2] = new_tokens[idx2], new_tokens[idx1]
    
    return new_tokens

def augment_text(text, augmentation_techniques=['synonym', 'insertion', 'deletion', 'swap'], 
                 num_augmentations=3):
    """Apply data augmentation to text"""
    augmented_texts = []
    
    for _ in range(num_augmentations):
        aug_text = text.copy()
        technique = random.choice(augmentation_techniques)
        
        if technique == 'synonym' and len(aug_text) > 0:
            aug_text = synonym_replacement(aug_text, n=random.randint(1, 2))
        elif technique == 'insertion' and len(aug_text) > 0:
            aug_text = random_insertion(aug_text, n=random.randint(1, 2))
        elif technique == 'deletion' and len(aug_text) > 1:
            aug_text = random_deletion(aug_text, p=0.1)
        elif technique == 'swap' and len(aug_text) > 1:
            aug_text = random_swap(aug_text, n=1)
        
        augmented_texts.append(aug_text)
    
    return augmented_texts

print("    ✓ Data augmentation functions ready")



>>> Step 2: Implementing Data Augmentation Functions...
    ✓ Data augmentation functions ready


In [31]:

# ============================================================================
# Step 3: Apply Data Augmentation for Imbalanced Classes
# ============================================================================

print("\n>>> Step 3: Applying Data Augmentation for Imbalanced Classes...")

# Count current label distribution
label_counts_p34 = Counter([ex.label for ex in train_data.examples])
print(f"\nOriginal label distribution:")
for label, count in sorted(label_counts_p34.items()):
    print(f"  {label}: {count} samples ({count/len(train_data.examples)*100:.2f}%)")

# Augmentation targets (boost weaker topics more aggressively)
target_counts_p34 = {
    'ABBR': 668,   # heavy boost to improve weakest class
    'DESC': 930,   # same
    'ENTY': 1500,  # same
    'HUM': int(978 * 1.15),   # same
    'LOC': int(668 * 1.15),    # same
    'NUM': int(717 * 1.15)     # same
}

# Create augmented examples
augmented_examples = list(train_data.examples)  # Start with all original examples

for label in label_counts_p34.keys():
    current_count = label_counts_p34[label]
    target_count = target_counts_p34[label]
    
    if current_count < target_count:
        label_examples = [ex for ex in train_data.examples if ex.label == label]
        num_augmentations_needed = target_count - current_count
        
        print(f"\n  Augmenting {label}: {current_count} -> {target_count} samples")
        print(f"    Generating {num_augmentations_needed} additional samples...")
        
        augmented_count = 0
        while augmented_count < num_augmentations_needed:
            original_ex = random.choice(label_examples)
            aug_texts = augment_text(original_ex.text, num_augmentations=1)
            
            for aug_text in aug_texts:
                if augmented_count >= num_augmentations_needed:
                    break
                
                new_ex = data.Example.fromlist([aug_text, label], 
                                               fields=[('text', TEXT), ('label', LABEL)])
                augmented_examples.append(new_ex)
                augmented_count += 1
        
        print(f"    ✓ Generated {augmented_count} augmented samples")

# Create augmented dataset with proper field structure
augmented_train_data = data.Dataset(augmented_examples, fields=[('text', TEXT), ('label', LABEL)])

# Verify augmented distribution
new_label_counts = Counter([ex.label for ex in augmented_examples])
print(f"\nAugmented label distribution:")
for label, count in sorted(new_label_counts.items()):
    print(f"  {label}: {count} samples ({count/len(augmented_examples)*100:.2f}%)")

print(f"\n  Total samples: {len(train_data.examples)} -> {len(augmented_examples)}")
print(f"  ✓ Data augmentation complete")


>>> Step 3: Applying Data Augmentation for Imbalanced Classes...

Original label distribution:
  ABBR: 69 samples (1.58%)
  DESC: 930 samples (21.32%)
  ENTY: 1000 samples (22.93%)
  HUM: 978 samples (22.42%)
  LOC: 668 samples (15.31%)
  NUM: 717 samples (16.44%)

  Augmenting LOC: 668 -> 768 samples
    Generating 100 additional samples...
    ✓ Generated 100 augmented samples

  Augmenting ENTY: 1000 -> 1500 samples
    Generating 500 additional samples...
    ✓ Generated 500 augmented samples

  Augmenting HUM: 978 -> 1124 samples
    Generating 146 additional samples...
    ✓ Generated 146 augmented samples

  Augmenting ABBR: 69 -> 668 samples
    Generating 599 additional samples...
    ✓ Generated 599 augmented samples

  Augmenting NUM: 717 -> 824 samples
    Generating 107 additional samples...
    ✓ Generated 107 augmented samples

Augmented label distribution:
  ABBR: 668 samples (11.49%)
  DESC: 930 samples (16.00%)
  ENTY: 1500 samples (25.80%)
  HUM: 1124 samples (19.33

In [32]:
# ============================================================================
# Step 3b: Refresh vocabulary and embeddings after augmentation
# ============================================================================

print("\n>>> Rebuilding TEXT/LABEL vocab with augmented data...")

# Rebuild vocabularies using augmented samples so new tokens get their own ids
TEXT.build_vocab(augmented_train_data, min_freq=1)
LABEL.build_vocab(augmented_train_data)

# Recreate the FastText-initialised embedding layer for the expanded vocab
augmented_embedding = embed_prep(TEXT)

print(f"  ✓ New vocab size: {len(TEXT.vocab)}")
print(f"  ✓ Embedding dim: {augmented_embedding.weight.shape[1]}")
print("  ✓ FastText embedding refreshed for augmented vocabulary")

# Downstream code expects `pretrained_embed` / `pretrained_embeddings`
pretrained_embed = augmented_embedding
pretrained_embeddings = augmented_embedding.weight.data.clone()



>>> Rebuilding TEXT/LABEL vocab with augmented data...
[*] Loading fasttext model...
[+] Model loaded!
[*] Forming embedding matrix...
[+] Embedding matrix formed!
  ✓ New vocab size: 8464
  ✓ Embedding dim: 300
  ✓ FastText embedding refreshed for augmented vocabulary


In [33]:
# augmented_train_data.examples[0].label
[(ex.text, ex.label) for ex in augmented_train_data if ex.label not in ['ABBR','DESC','ENTY','HUM','LOC','NUM']]

[]

In [34]:
LABEL.vocab.stoi

defaultdict(None,
            {'ENTY': 0, 'HUM': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})

In [35]:
abbr_aug_ex = [ex for ex in augmented_train_data if ex.label == "ABBR"]
abbr_ex = [ex for ex in train_data if ex.label == "ABBR"]

count = 0
# for ex in abbr_aug_ex:
#     if ex not in abbr_ex:
#         print(ex.text)
#         print(ex.label)
#         count += 1

print(count)

0


In [36]:
# Count how many samples per label in the train set
label_counts_p34 = Counter([ex.label for ex in augmented_train_data.examples])
total_examples_p34 = len(augmented_train_data)

print("\nLabel distribution in training set:")
for label, count in sorted(label_counts_p34.items()):
    percentage = (count / total_examples_p34) * 100
    print(f"- {label}: {count} samples ({percentage:.2f}%)")


Label distribution in training set:
- ABBR: 668 samples (11.49%)
- DESC: 930 samples (16.00%)
- ENTY: 1500 samples (25.80%)
- HUM: 1124 samples (19.33%)
- LOC: 768 samples (13.21%)
- NUM: 824 samples (14.17%)


In [37]:
# ============================================================================
# PART 3.5: Sampling Strategies vs Data Augmentation
# ============================================================================

print("\n" + "="*80)
print("PART 3.4: SAMPLING STRATEGIES VS DATA AUGMENTATION")
print("="*80)
print("Comparing text augmentation, weighted sampling, and their combination")
print("across the Simple RNN baseline (Part 2 best config) and the RNN + BERT hybrid.")
print("="*80)

# Extended RNN Classifier with multiple aggregation methods
class RNN_Classifier_Aggregation(nn.Module):
    """
    RNN for topic classification with multiple aggregation strategies.
    Uses pretrained embeddings (learnable/updated during training).
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers=1, dropout=0.0, padding_idx=0, pretrained_embeddings=None,
                 aggregation='mean'):
        super(RNN_Classifier_Aggregation, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.aggregation = aggregation  # 'last', 'mean', 'max'
        
        # Embedding layer initialized with pretrained embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            self.embedding.weight.requires_grad = True
        
        # RNN layer
        self.rnn = nn.RNN(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.0  # No dropout in baseline
        )
                
        # Fully connected output layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):        
        # Embed the input
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
        
        # Get dimensions
        batch_size = embedded.size(0)
        seq_len = embedded.size(1)
        
        # Handle text_lengths
        text_lengths_flat = text_lengths.flatten().cpu().long()
        if len(text_lengths_flat) != batch_size:
            raise ValueError(
                f"text_lengths size mismatch: got {len(text_lengths_flat)} elements, "
                f"expected {batch_size}"
            )
        
        # Clamp lengths
        text_lengths_clamped = torch.clamp(text_lengths_flat, min=1, max=seq_len)
        
        text_lengths_clamped_device = text_lengths_clamped.to(text.device)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths_clamped, batch_first=True, enforce_sorted=False
        )
        
        # Pass through RNN
        packed_output, hidden = self.rnn(packed_embedded)
        
        # Aggregate word representations to sentence representation
        if self.aggregation == 'last':
            sentence_repr = hidden[-1]  # [batch_size, hidden_dim]
            
        elif self.aggregation == 'mean':
            output, output_lengths = nn.utils.rnn.pad_packed_sequence(
                packed_output, batch_first=True
            )
            # output: [batch_size, seq_len, hidden_dim]
            
            # Create mask for padding
            mask = torch.arange(seq_len, device=text.device).unsqueeze(0) < text_lengths_clamped_device.unsqueeze(1)
            mask = mask.unsqueeze(2).float()  # [batch_size, seq_len, 1]
            
            # Apply mask and compute mean
            masked_output = output * mask
            sum_output = masked_output.sum(dim=1)  # [batch_size, hidden_dim]
            sentence_repr = sum_output / text_lengths_clamped_device.unsqueeze(1).float()
            
        elif self.aggregation == 'max':
            # Max pooling over all outputs
            output, output_lengths = nn.utils.rnn.pad_packed_sequence(
                packed_output, batch_first=True
            )
            # output: [batch_size, seq_len, hidden_dim]
            
            mask = torch.arange(seq_len, device=text.device).unsqueeze(0) < text_lengths_clamped_device.unsqueeze(1)
            mask = mask.unsqueeze(2).float()  # [batch_size, seq_len, 1]
            
            masked_output = output * mask + (1 - mask) * float('-inf')
            sentence_repr, _ = torch.max(masked_output, dim=1)
        
        # Pass through fully connected layer
        output = self.fc(sentence_repr)  # [batch_size, output_dim]
        
        return output

print(">>> Simple RNN ready (mean aggregation baseline)")



PART 3.4: SAMPLING STRATEGIES VS DATA AUGMENTATION
Comparing text augmentation, weighted sampling, and their combination
across the Simple RNN baseline (Part 2 best config) and the RNN + BERT hybrid.
>>> Simple RNN ready (mean aggregation baseline)


In [38]:
# =========================================================================
# Helper: Topic-wise evaluation
# =========================================================================

def evaluate_per_topic_p35(model, iterator, device, text_vocab=None):
    """Evaluate accuracy per topic on the provided iterator."""
    model.eval()
    topic_correct = defaultdict(int)
    topic_total = defaultdict(int)
    idx_to_label = LABEL.vocab.itos

    with torch.no_grad():
        for batch in iterator:
            text, text_lengths, labels = process_batch(batch, debug=False)
            if text_vocab is not None:
                predictions = model(text, text_lengths, text_vocab=text_vocab)
            else:
                predictions = model(text, text_lengths)
            preds = torch.argmax(predictions, dim=1)
            for pred, label in zip(preds.cpu().numpy(), labels.cpu().numpy()):
                topic_name = idx_to_label[label]
                topic_total[topic_name] += 1
                if pred == label:
                    topic_correct[topic_name] += 1

    topic_metrics = {}
    for topic in sorted(topic_total.keys()):
        total = topic_total[topic]
        correct = topic_correct[topic]
        accuracy = correct / total if total > 0 else 0.0
        topic_metrics[topic] = {
            "accuracy": accuracy,
            "correct": correct,
            "total": total,
        }
    return topic_metrics

In [39]:
# ============================================================================
# Dataset Variants & Utilities for Experiments
# ============================================================================

def describe_dataset(name, dataset):
    counts = Counter(ex.label for ex in dataset.examples)
    total = len(dataset.examples)
    print(f"  - {name}: {total} samples")
    for label, count in sorted(counts.items()):
        print(f"      {label}: {count} ({count/total*100:.2f}%)")
    return counts


# Topic-wise accuracy from latest weighted-sampler evaluation (used to boost weak classes)
P35_TOPIC_ACCURACY = {
    "ABBR": 0.7778,
    "DESC": 0.9855,
    "ENTY": 0.7128,
    "HUM": 0.8769,
    "LOC": 0.8889,
    "NUM": 0.8584,
}
# Convert to difficulty scores (higher when accuracy is lower)
P35_TOPIC_DIFFICULTY = {label: max(0.0, 1.0 - acc) for label, acc in P35_TOPIC_ACCURACY.items()}
# Global multiplier for difficulty adjustment; tweak to emphasise weak topics more/less
P35_DIFFICULTY_SCALE = 2.0


def create_weighted_dataset(source_dataset, target_size=None, seed=SEED, difficulty_scale=P35_DIFFICULTY_SCALE):
    """Mimic WeightedRandomSampler by sampling examples according to class weights, with extra boosts for weak topics."""
    rng = random.Random(seed)
    counts = Counter(ex.label for ex in source_dataset.examples)
    total = sum(counts.values())
    base_class_weights = {label: total / count for label, count in counts.items()}

    class_boosts = {
        label: 1.0 + difficulty_scale * P35_TOPIC_DIFFICULTY.get(label, 0.0)
        for label in counts.keys()
    }

    weights = [base_class_weights[ex.label] * class_boosts.get(ex.label, 1.0) for ex in source_dataset.examples]
    sample_size = target_size or len(source_dataset.examples)

    sampled_examples = rng.choices(source_dataset.examples, weights=weights, k=sample_size)
    fields = [('text', TEXT), ('label', LABEL)]
    return data.Dataset(sampled_examples, fields=fields)


print("\n>>> Preparing dataset variants for Part 3.4 experiments...")
base_counts = describe_dataset("Original train", train_data)
aug_counts = describe_dataset("Augmented train", augmented_train_data)

weighted_train_data = create_weighted_dataset(train_data)
weighted_counts = describe_dataset("Weighted-sampled train", weighted_train_data)

augmented_weighted_train_data = create_weighted_dataset(augmented_train_data)
aug_weighted_counts = describe_dataset("Augmented + weighted train", augmented_weighted_train_data)

p35_datasets = {
    "original": train_data,
    "augmented": augmented_train_data,
    "weighted": weighted_train_data,
    "augmented_weighted": augmented_weighted_train_data,
}

criterion = nn.CrossEntropyLoss()

p35_results = {
    "simple_rnn_baseline": {},
    "rnn_bert": {}
}

print("\n>>> Dataset variants ready. Criterion initialised for upcoming runs.")



>>> Preparing dataset variants for Part 3.4 experiments...
  - Original train: 4362 samples
      ABBR: 69 (1.58%)
      DESC: 930 (21.32%)
      ENTY: 1000 (22.93%)
      HUM: 978 (22.42%)
      LOC: 668 (15.31%)
      NUM: 717 (16.44%)
  - Augmented train: 5814 samples
      ABBR: 668 (11.49%)
      DESC: 930 (16.00%)
      ENTY: 1500 (25.80%)
      HUM: 1124 (19.33%)
      LOC: 768 (13.21%)
      NUM: 824 (14.17%)
  - Weighted-sampled train: 4362 samples
      ABBR: 843 (19.33%)
      DESC: 547 (12.54%)
      ENTY: 884 (20.27%)
      HUM: 725 (16.62%)
      LOC: 667 (15.29%)
      NUM: 696 (15.96%)
  - Augmented + weighted train: 5814 samples
      ABBR: 1041 (17.91%)
      DESC: 769 (13.23%)
      ENTY: 1156 (19.88%)
      HUM: 934 (16.06%)
      LOC: 901 (15.50%)
      NUM: 1013 (17.42%)

>>> Dataset variants ready. Criterion initialised for upcoming runs.


In [40]:
# ============================================================================
# Simple RNN (mean pooling) experiment runner
# ============================================================================

def reset_random_seeds(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def build_iterator(dataset, batch_size, shuffle):
    return data.BucketIterator(
        dataset,
        batch_size=batch_size,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=shuffle,
        device=device
    )

RNN_BASE_HYPERPARM = hyperparam_prep()
RNN_BASE_HYPERPARM['HIDDEN_DIM'] *= 2
RNN_BASE_HYPERPARM['N_LAYERS'] = 1
RNN_BASE_HYPERPARM['SAVE_MODEL'] = True


def run_simple_rnn_experiment(dataset_key, description, save_suffix):
    if dataset_key not in p35_datasets:
        raise ValueError(f"Unknown dataset key: {dataset_key}")

    reset_random_seeds(SEED)
    train_dataset = p35_datasets[dataset_key]

    print("\n" + "-"*80)
    print(f"Running Simple RNN (mean pooling) experiment: {description}")
    print("-"*80)

    train_iter = build_iterator(train_dataset, RNN_BASE_HYPERPARM['BATCH_SIZE'], shuffle=True)
    val_iter = build_iterator(validation_data, RNN_BASE_HYPERPARM['BATCH_SIZE'], shuffle=False)
    test_iter = build_iterator(test_data, RNN_BASE_HYPERPARM['BATCH_SIZE'], shuffle=False)

    model = RNN_Classifier_Aggregation(
        vocab_size=len(TEXT.vocab),
        embedding_dim=embedding_dim,
        hidden_dim=RNN_BASE_HYPERPARM['HIDDEN_DIM'],
        output_dim=num_classes,
        n_layers=RNN_BASE_HYPERPARM['N_LAYERS'],
        dropout=RNN_BASE_HYPERPARM['DROPOUT'],
        padding_idx=TEXT.vocab.stoi[TEXT.pad_token],
        pretrained_embeddings=pretrained_embeddings,
        aggregation=RNN_BASE_HYPERPARM['AGGREGATOR'],
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    
    # Select optimizer with best learning rate
    if RNN_BASE_HYPERPARM['OPTIMIZER'] == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=RNN_BASE_HYPERPARM['LEARNING_RATE'],
                                        weight_decay=RNN_BASE_HYPERPARM['L2_LAMBDA'])
    elif RNN_BASE_HYPERPARM['OPTIMIZER'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=RNN_BASE_HYPERPARM['LEARNING_RATE'], momentum=0.9,
                                        weight_decay=RNN_BASE_HYPERPARM['L2_LAMBDA'])
    elif RNN_BASE_HYPERPARM['OPTIMIZER'] == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=RNN_BASE_HYPERPARM['LEARNING_RATE'],
                                        weight_decay=RNN_BASE_HYPERPARM['L2_LAMBDA'])
    elif RNN_BASE_HYPERPARM['OPTIMIZER'] == 'Adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=RNN_BASE_HYPERPARM['LEARNING_RATE'],
                                        weight_decay=RNN_BASE_HYPERPARM['L2_LAMBDA'])

    model, history = train_model_with_history(
        model,
        train_iter,
        val_iter,
        optimizer,
        criterion,
        RNN_BASE_HYPERPARM['N_EPOCHS'],
        device,
        num_classes,
        RNN_BASE_HYPERPARM['L1_LAMBDA'],
        patience=RNN_BASE_HYPERPARM['PATIENCE'],
        model_name=f"Simple RNN ({description})",
    )

    test_loss, test_acc, test_f1, test_auc = evaluate_model(
        model,
        test_iter,
        criterion,
        device,
        f"Simple RNN ({description})",
        num_classes,
    )

    topic_metrics = evaluate_per_topic_p35(model, test_iter, device)

    model_path = f"weights/part35_simple_rnn_{save_suffix}.pt"
    if RNN_BASE_HYPERPARM['SAVE_MODEL']:
        torch.save(model.state_dict(), model_path)

    p35_results["simple_rnn_baseline"][save_suffix] = {
        "description": description,
        "dataset_key": dataset_key,
        "history": history,
        "test_metrics": {
            "loss": test_loss,
            "accuracy": test_acc,
            "f1": test_f1,
            "auc": test_auc,
        },
        "topic_metrics": topic_metrics,
        "model_path": model_path if RNN_BASE_HYPERPARM['SAVE_MODEL'] else None,
        "model": model,
    }

    return {
        "history": history,
        "topic_metrics": topic_metrics,
        "test_metrics": {
            "loss": test_loss,
            "accuracy": test_acc,
            "f1": test_f1,
            "auc": test_auc,
        },
    }

In [41]:
# ============================================================================
# Run Simple RNN + Attention experiments
# ============================================================================

print("\n>>> Executing Simple RNN experiments...")

rnn_results_text_aug = run_simple_rnn_experiment(
    dataset_key="augmented",
    description="Text Augmentation",
    save_suffix="text_aug",
)

rnn_results_weighted = run_simple_rnn_experiment(
    dataset_key="weighted",
    description="Weighted Sampling",
    save_suffix="weighted_sampler",
)

rnn_results_aug_weighted = run_simple_rnn_experiment(
    dataset_key="augmented_weighted",
    description="Augmentation + Weighted Sampling",
    save_suffix="text_aug_weighted",
)

print("\n>>> Simple RNN experiments queued. Run the cells to execute training if needed.")



>>> Executing Simple RNN experiments...

--------------------------------------------------------------------------------
Running Simple RNN (mean pooling) experiment: Text Augmentation
--------------------------------------------------------------------------------



>>> Training Simple RNN (Text Augmentation)
    Parameters: 2,959,046
    Max epochs: 100, Patience: 10
Epoch: 01/100 | Time: 0m 3s
	Train Loss: 1.7102 | Train Acc: 31.03%
	Val Loss: 1.3900 | Val Acc: 45.14% | Val F1: 0.3828 | Val AUC: 0.7963
Epoch: 02/100 | Time: 0m 2s
	Train Loss: 1.1816 | Train Acc: 59.67%
	Val Loss: 0.9560 | Val Acc: 65.78% | Val F1: 0.6629 | Val AUC: 0.8946
Epoch: 03/100 | Time: 0m 3s
	Train Loss: 0.8139 | Train Acc: 77.64%
	Val Loss: 0.7732 | Val Acc: 73.85% | Val F1: 0.7456 | Val AUC: 0.9264
Epoch: 04/100 | Time: 0m 3s
	Train Loss: 0.6030 | Train Acc: 85.55%
	Val Loss: 0.6560 | Val Acc: 79.08% | Val F1: 0.7977 | Val AUC: 0.9460
Epoch: 05/100 | Time: 0m 3s
	Train Loss: 0.4788 | Train Acc: 90.08%
	Val Loss: 0.5941 | Val Acc: 82.20% | Val F1: 0.8238 | Val AUC: 0.9546
Epoch: 06/100 | Time: 0m 3s
	Train Loss: 0.4086 | Train Acc: 91.25%
	Val Loss: 0.5726 | Val Acc: 83.03% | Val F1: 0.8331 | Val AUC: 0.9591
Epoch: 07/100 | Time: 0m 3s
	Train Loss: 0.3323 | Train Acc: 

In [42]:
# ============================================================================
# Topic-wise accuracy summary for Simple RNN experiments
# ============================================================================

def display_topic_metrics(title, metrics_dict):
    print(f"\n{title}")
    print("-" * len(title))
    header = f"{'Topic':<10} {'Accuracy %':<12} {'Correct':<10} {'Total':<10}"
    print(header)
    print("-" * len(header))
    for topic in sorted(metrics_dict.keys()):
        stats = metrics_dict[topic]
        acc_pct = stats['accuracy'] * 100
        print(f"{topic:<10} {acc_pct:<12.2f} {stats['correct']:<10} {stats['total']:<10}")
    print("-" * len(header))
    
    total_cor = sum([metrics_dict[topic]['correct'] for topic in metrics_dict])
    total_sam = sum([metrics_dict[topic]['total'] for topic in metrics_dict])
    total_acc = total_cor / total_sam 
    print(f"{'Topic':<10} {total_acc:<12.2f} {total_cor:<10} {total_sam:<10}")

print("\n>>> Topic-wise accuracy for Simple RNN variants")
for run_key, info in p35_results["simple_rnn_baseline"].items():
    topic_metrics = info.get("topic_metrics")
    if not topic_metrics:
        continue
    title = f"Simple RNN ({info['description']})"
    display_topic_metrics(title, topic_metrics)


>>> Topic-wise accuracy for Simple RNN variants

Simple RNN (Text Augmentation)
------------------------------
Topic      Accuracy %   Correct    Total     
---------------------------------------------
ABBR       77.78        7          9         
DESC       97.83        135        138       
ENTY       71.28        67         94        
HUM        89.23        58         65        
LOC        81.48        66         81        
NUM        84.07        95         113       
---------------------------------------------
Topic      0.86         428        500       

Simple RNN (Weighted Sampling)
------------------------------
Topic      Accuracy %   Correct    Total     
---------------------------------------------
ABBR       77.78        7          9         
DESC       97.83        135        138       
ENTY       64.89        61         94        
HUM        86.15        56         65        
LOC        82.72        67         81        
NUM        84.07        95         113     